In [1]:
!pip install pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached autograd-1.6.2-py3-none-any.whl.metadata (706 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 7.5 MB/s eta 0:00:00
  Using cached future-0.18.3.tar.gz (840 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using cached autograd-1.6.2-py3-none-any.whl (49 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
  Created wheel for pymoo: filename=pymoo-0.6.1.1-py3-none-any.whl size=329948 sha256=2681a61e40de4db8e490cf2f38f8b88e9750acca4213cc3d3ab33c9fff2c0cb8
  Stored in directory: /Users/kemalbayik/Library/Caches/pip/wheels/ce/6

In [2]:
!pip install pymoode==0.1.8

  Using cached pymoode-0.1.8-py3-none-any.whl (25 kB)
  Using cached pymoo-0.5.0.tar.gz (706 kB)
  Preparing metadata (setup.py) ... done
  Using cached cma-2.7.0-py2.py3-none-any.whl (239 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 40.9 MB/s eta 0:00:0000:0100:01
  Created wheel for pymoo: filename=pymoo-0.5.0-cp38-cp38-macosx_11_0_arm64.whl size=1112144 sha256=92aa6c82032ace7846ab6e4e1c9583a2496e59ba9466596d3128492b46eb866a
  Stored in directory: /Users/kemalbayik/Library/Caches/pip/wheels/d5/a0/39/1d2f8d321237f72a9f4e1bf1ba287cf0ab381c5efbffd8be2b
Successfully built pymoo
DEPRECATION: pymoode 0.1.8 has a non-standard dependency specifier numpy>=1.19.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pymoode or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pymoode 0.1.8 has a non-st

In [3]:
from numpy import random
from pymoo.core.problem import Problem
from pymoo.core.problem import ElementwiseProblem
import matplotlib.pyplot as plt
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.pcx import PCX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize
from pymoode.gde3 import GDE3
from pymoo.factory import get_performance_indicator
from sklearn.preprocessing import MinMaxScaler
import time
import numpy as np

In [4]:
class Dataset:

  def __init__(self, number_of_facilities, number_of_stores, fixed_costs_for_facilities, number_of_cases_capacity, number_of_stores_capacity, store_demands, transportation_cost, total_transport_co2_emissions):
    """
           Constructor of Dataset class
        :param number_of_facilities: Number of facilities
        :param number_of_stores: Number of stores
        :param fixed_costs_for_facilities: Fixed costs of facilities
        :param number_of_cases_capacity: Case capacities of facilities
        :param number_of_stores_capacity: Store capacities of facilities
        :param store_demands: Demands of stores
        :param transportation_cost: Transportation costs
        :param total_transport_co2_emissions: Total CO2 emissions of transportations
    """
    self.number_of_facilities = number_of_facilities
    self.number_of_stores = number_of_stores
    self.fixed_costs_for_facilities = fixed_costs_for_facilities
    self.number_of_cases_capacity = number_of_cases_capacity
    self.number_of_stores_capacity = number_of_stores_capacity
    self.store_demands = store_demands
    self.transportation_cost = transportation_cost # Transportation cost to satisfy all demand from each facility to each store
    self.total_transport_co2_emissions = total_transport_co2_emissions  # Total transport CO2 emissions to satisfy all demand from each facility to each store

In [5]:
class MOCFLP(ElementwiseProblem):
    def __init__(self, number_of_facilities, number_of_stores, fixed_costs_for_facilities, number_of_stores_capacity, number_of_cases_capacity, store_demands, transportation_cost, total_transport_co2_emissions):
        """
           Constructor of MOCFLP problem implementation class
        :param number_of_facilities: Number of facilities
        :param number_of_stores: Number of stores
        :param fixed_costs_for_facilities: Fixed costs of facilities
        :param number_of_cases_capacity: Case capacities of facilities
        :param number_of_stores_capacity: Store capacities of facilities
        :param store_demands: Demands of stores
        :param transportation_cost: Transportation costs
        :param total_transport_co2_emissions: Total CO2 emissions of transportations
        """
        super().__init__(n_var=number_of_stores,
                         n_obj=2,
                         n_constr=2 * number_of_facilities,
                         xl=0,
                         xu=number_of_facilities - 1)
        self.number_of_facilities = number_of_facilities
        self.number_of_stores = number_of_stores
        self.fixed_costs_for_facilities = fixed_costs_for_facilities
        self.transportation_cost = transportation_cost
        self.total_transport_co2_emissions = total_transport_co2_emissions
        self.number_of_stores_capacity = number_of_stores_capacity
        self.number_of_cases_capacity = number_of_cases_capacity
        self.store_demands = store_demands

    def _evaluate(self, x, out, *args, **kwargs):
        x = np.round(x).astype(int)
        total_cost = 0
        total_co2 = 0
        facilities_used = set(x)

        #Evaluating transportation cost and co2 emissions
        for i in range(self.number_of_stores):
            facility_idx = x[i]
            total_cost += self.transportation_cost[i][facility_idx] * self.store_demands[i]
            total_co2 += self.total_transport_co2_emissions[i][facility_idx] * self.store_demands[i]

        #Adding used facilities fixed costs to find the total cost
        for facility in facilities_used:
            total_cost += self.fixed_costs_for_facilities[facility]

        G_store = np.array([np.sum(x == j) - self.number_of_stores_capacity[j] for j in range(self.number_of_facilities)])
        G_case = np.array([np.sum(self.store_demands[x == j]) - self.number_of_cases_capacity[j] for j in range(self.number_of_facilities)])

        out["F"] = np.array([total_cost, total_co2])
        out["G"] = np.concatenate([G_store, G_case])

In [102]:
class Algorithms:

  def __init__(self):
    pass

  def nsga2_algorithm(self, problem, number_of_generations, population_size, crossover_rate, show_plot):

      """
            NSGA-II algorithm implementation
        :param problem: MOCFLP class object
        :param number_of_generations: Number of generations
        :param population_size: Population size
        :param crossover_rate: Crossover rate
        :param show_plot: Prints pareto front plot to the screen if True
        :return: Sorted pareto front results
        :return: Best facility indices
        :return: Runtime of the algorithm
      """
      # Starting time to find the runtime
      start_time = time.time()
      
      # Algorithm setup
      algorithm = NSGA2(
          pop_size=population_size,
          sampling=FloatRandomSampling(),
          crossover=PCX(prob=crossover_rate),
          mutation=PM(eta=0.7),
          eliminate_duplicates=True
      )

      # Solve the problem
      res = minimize(
          problem,
          algorithm,
          ('n_gen', number_of_generations),  # Termination criterion
          seed=1,
          save_history=True,
          verbose=False
      )

      best_solution = res.X[np.argmin(res.F[:,0])]
      best_facility_indices = np.round(best_solution).astype(int)

      # Finding open facilities
      open_facilities = np.unique(best_facility_indices)

      #print("Open Facilities:", open_facilities)
      #for customer in range(problem.number_of_stores):
      #    print(f"Customer {customer} is served by Facility {best_facility_indices[customer]}")

      # Sorting pareto front
      sorted_final_front_nsga2 = res.F[np.argsort(res.F[:, 0])]

      # If show_plot is True prints pareto front plot
      if show_plot:

        plt.figure(figsize=(10, 6))
        colors = plt.cm.viridis(np.linspace(0, 1, len(res.history)))

        for i, entry in enumerate(res.history):
          plt.scatter(entry.pop.get("F")[:, 0], entry.pop.get("F")[:, 1], s=12, facecolors='none', edgecolors=colors[i], label=f'Gen {i+1}' if i % 20 == 0 else "")
        # Pareto front
        plt.scatter(sorted_final_front_nsga2[:, 0], sorted_final_front_nsga2[:, 1], c='red', label='Final Pareto Front')

        plt.plot(sorted_final_front_nsga2[:, 0], sorted_final_front_nsga2[:, 1], c='red')

        plt.title('NSGA-II Progression of Pareto Front')
        plt.xlabel('Total Cost')
        plt.ylabel('Total CO2 Emissions')
        plt.grid(True)
        plt.legend()
        plt.show()

      # Finding the runtime
      runtime = time.time() - start_time

      return sorted_final_front_nsga2, best_facility_indices, runtime


  def gde3_algorithm(self, problem, number_of_generations, population_size, crossover_rate, show_plot):
      """
            GDE3 algorithm implementation
        :param problem: MOCFLP class object
        :param number_of_generations: Number of generations
        :param population_size: Population size
        :param crossover_rate: Crossover rate
        :param show_plot: Prints pareto front plot to the screen if True
        :return: Sorted pareto front results
        :return: Best facility indices
        :return: Runtime of the algorithm
      """

      # Starting time to find the runtime
      start_time = time.time()

      # Algorithm setup
      gde3 = GDE3(pop_size=population_size, variant="DE/rand/1/bin", CR=crossover_rate, F=0.7)

      res = minimize(problem, gde3, ('n_gen', number_of_generations), save_history=True, verbose=False)

      best_solution = res.X[np.argmin(res.F[:,0])]
      best_facility_indices = np.round(best_solution).astype(int)

      # Finding open facilities
      open_facilities = np.unique(best_facility_indices)

      # Sorting pareto front
      sorted_final_front_gde3 = res.F[np.argsort(res.F[:, 0])]


      if show_plot:

        # Plotting the Pareto front progression
        plt.figure(figsize=(10, 6))
        colors = plt.cm.viridis(np.linspace(0, 1, len(res.history)))

        for i, entry in enumerate(res.history):
          plt.scatter(entry.pop.get("F")[:, 0], entry.pop.get("F")[:, 1], s=12, facecolors='none', edgecolors=colors[i], label=f'Gen {i+1}' if i % 20 == 0 else "")
        # Pareto front
        plt.scatter(sorted_final_front_gde3[:, 0], sorted_final_front_gde3[:, 1], c='red', label='Final Pareto Front')
    
        plt.plot(sorted_final_front_gde3[:, 0], sorted_final_front_gde3[:, 1], c='red')
    
        plt.title('GDE3 Progression of Pareto Front')
        plt.xlabel('Total Cost')
        plt.ylabel('Total CO2 Emissions')
        plt.grid(True)
        plt.legend()
        plt.show()

      # Finding the runtime
      runtime = time.time() - start_time

      return sorted_final_front_gde3, best_facility_indices, runtime


  def calculate_hypervolume(self, pareto_front_gde3, pareto_front_nsga2):

      """
            Calculates hypervolume of GDE3's and NSGA-II's pareto front results
        :param pareto_front_gde3: Pareto front results of GDE3
        :param pareto_front_nsga2: Pareto front results of NSGA-II
        :return: Hypervolume of GDE3's pareto front results
        :return: Hypervolume of NSGA-II's pareto front results
      """

      reference_point = np.array([1.1, 1.1])  # Reference point is 1.1 because I scale the data
      hv = get_performance_indicator("hv", ref_point=reference_point)

      scaler = MinMaxScaler()
      scaled_pareto_front_gde3 = scaler.fit_transform(pareto_front_gde3)
      scaled_pareto_front_nsga2 = scaler.fit_transform(pareto_front_nsga2)

      hv_gde3 = hv.do(scaled_pareto_front_gde3)
      hv_nsga2 = hv.do(scaled_pareto_front_nsga2)

      return hv_gde3,hv_nsga2

  def spacing(self, pareto_front):
    """
        Calculates spacing value of given pareto front result
    :param pareto_front: Pareto front result
    :return: Spacing value of given pareto front result
    """
    n = len(pareto_front)

    if n < 2:
        return 0

    distances = np.zeros(n)
    for i in range(n):
        min_dist = np.inf
        for j in range(n):
            if i != j:
                dist = np.linalg.norm(pareto_front[i] - pareto_front[j])
                min_dist = min(min_dist, dist)
        distances[i] = min_dist

    mean_distance = np.mean(distances)
    spacing_value = np.sqrt(np.sum((distances - mean_distance)**2) / n)
    return spacing_value


  def experiment(self, number_of_facilities, number_of_stores, fixed_costs_for_facilities, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot):

        
        """
            Runs the experiment according to given parameters
        :param number_of_facilities: Number of facilities
        :param number_of_stores: Number of stores
        :param fixed_costs_for_facilities: Fixed costs of facilities
        :param number_of_cases_capacity: Case capacities of facilities
        :param number_of_stores_capacity: Store capacities of facilities
        :param demand_for_stores: Demands of stores
        :param transportation_cost: Transportation costs
        :param total_transport_co2_emissions: Total CO2 emissions of transportations
        :param num_iterations: Number of iterations
        :param number_of_generations: Number of generations
        :param population_size: Population size
        :param crossover_rate: Crossover rate
        :param show_plot: Prints pareto front plot to the screen if True
        """
        
        dataset = Dataset(number_of_facilities, number_of_stores, fixed_costs_for_facilities,
                             number_of_cases_capacity, number_of_stores_capacity, demand_for_stores,
                             transportation_cost, total_transport_co2_emissions)

        problem = MOCFLP(dataset.number_of_facilities, dataset.number_of_stores, dataset.fixed_costs_for_facilities, 
                         dataset.number_of_stores_capacity, dataset.number_of_cases_capacity, dataset.store_demands, 
                         dataset.transportation_cost, dataset.total_transport_co2_emissions)
        
        runtimes_gde3 = []
        runtimes_nsga2 = []
        hypervolumes_gde3 = []
        hypervolumes_nsga2 = []
        spacing_values_gde3 = []
        spacing_values_nsga2 = []
        coverage_values_gde3 = []
        coverage_values_nsga2 = []
        
        for i in range(num_iterations):
        
          print("Iteration : ",str(i))
        
          gde3_pareto_fronts, gde3_best_facility_indices, gde3_runtime = self.gde3_algorithm(problem, number_of_generations, 
                                                                                                   population_size, crossover_rate, show_plot)
          nsga2_pareto_fronts, nsga2_best_facility_indices, nsga2_runtime = self.nsga2_algorithm(problem, number_of_generations, 
                                                                                                       population_size, crossover_rate, show_plot)
        
          runtimes_gde3.append(gde3_runtime)
          runtimes_nsga2.append(nsga2_runtime)
        
          hypervolume_gde3, hypervolume_nsga2 = self.calculate_hypervolume(gde3_pareto_fronts,nsga2_pareto_fronts)
        
          hypervolumes_gde3.append(hypervolume_gde3)
          hypervolumes_nsga2.append(hypervolume_nsga2)
        
        
          spacing_gde3 = self.spacing(gde3_pareto_fronts)
          spacing_nsga2 = self.spacing(nsga2_pareto_fronts)
        
          spacing_values_gde3.append(spacing_gde3)
          spacing_values_nsga2.append(spacing_nsga2)
        
        print("GDE3 Average Runtime ",str(sum(runtimes_gde3) / num_iterations))
        print("NSGA-II Average Runtime ",str(sum(runtimes_nsga2) / num_iterations))
        
        print("GDE3 Average Hypervolume ",str(sum(hypervolumes_gde3) / num_iterations))
        print("NSGA-II Average Hypervolume ",str(sum(hypervolumes_nsga2) / num_iterations))
        
        print("GDE3 Average Spacing ",str(sum(spacing_values_gde3) / num_iterations))
        print("NSGA-II Average Spacing ",str(sum(spacing_values_nsga2) / num_iterations))
        

In [76]:
number_of_facilities = 5
number_of_stores = 30
fixed_costs_for_warehouses = [10000, 10000, 10000, 10000, 10000]
number_of_cases_capacity = [20000, 20000, 20000, 20000, 20000]
number_of_stores_capacity = [10, 10, 10, 10, 10]
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 100, 100, 0.7, False

result_1_100_100_100_07 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [77]:
number_of_facilities = 5
number_of_stores = 30
fixed_costs_for_warehouses = [10000, 10000, 10000, 10000, 10000]
number_of_cases_capacity = [20000, 20000, 20000, 20000, 20000]
number_of_stores_capacity = [10, 10, 10, 10, 10]
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 100, 200, 0.7, False

result_1_100_100_200_07 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [78]:
number_of_facilities = 5
number_of_stores = 30
fixed_costs_for_warehouses = [10000, 10000, 10000, 10000, 10000]
number_of_cases_capacity = [20000, 20000, 20000, 20000, 20000]
number_of_stores_capacity = [10, 10, 10, 10, 10]
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 100, 500, 0.7, False

result_1_100_100_500_07 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [79]:
number_of_facilities = 5
number_of_stores = 30
fixed_costs_for_warehouses = [10000, 10000, 10000, 10000, 10000]
number_of_cases_capacity = [20000, 20000, 20000, 20000, 20000]
number_of_stores_capacity = [10, 10, 10, 10, 10]
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 100, 100, 0.3, False

result_1_100_100_100_03 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [80]:
number_of_facilities = 5
number_of_stores = 30
fixed_costs_for_warehouses = [10000, 10000, 10000, 10000, 10000]
number_of_cases_capacity = [20000, 20000, 20000, 20000, 20000]
number_of_stores_capacity = [10, 10, 10, 10, 10]
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 100, 100, 0.9, False

result_1_100_100_100_09 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [81]:
number_of_facilities = 5
number_of_stores = 30
fixed_costs_for_warehouses = [10000, 10000, 10000, 10000, 10000]
number_of_cases_capacity = [20000, 20000, 20000, 20000, 20000]
number_of_stores_capacity = [10, 10, 10, 10, 10]
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 200, 100, 0.7, False

result_1_100_200_100_07 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [82]:
number_of_facilities = 5
number_of_stores = 30
fixed_costs_for_warehouses = [10000, 10000, 10000, 10000, 10000]
number_of_cases_capacity = [20000, 20000, 20000, 20000, 20000]
number_of_stores_capacity = [10, 10, 10, 10, 10]
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 500, 100, 0.7, False

result_1_100_500_100_07 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [83]:
number_of_facilities = 20
number_of_stores = 500
fixed_costs_for_warehouses = np.empty(number_of_facilities)
fixed_costs_for_warehouses = random.randint(low = 5000, high = 10000, size=(number_of_stores))
number_of_cases_capacity = np.empty(number_of_facilities)
number_of_cases_capacity.fill(25000)
number_of_stores_capacity = np.empty(number_of_facilities)
number_of_stores_capacity.fill(30)
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 100, 100, 0.7, False

result_2_100_100_100_07 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It

In [84]:
number_of_facilities = 50
number_of_stores = 1000
fixed_costs_for_warehouses = np.empty(number_of_facilities)
fixed_costs_for_warehouses = random.randint(low = 5000, high = 10000, size=(number_of_stores))
number_of_cases_capacity = np.empty(number_of_facilities)
number_of_cases_capacity.fill(50000)
number_of_stores_capacity = np.empty(number_of_facilities)
number_of_stores_capacity.fill(30)
demand_for_stores = random.randint(low = 500, high = 1000, size=(number_of_stores))
transportation_cost = []

for i in range(number_of_stores):
  cost = random.randint(low = 2000, high = 20000, size=(number_of_facilities))
  transportation_cost.append(cost.tolist())

total_transport_co2_emissions = []

for i in range(number_of_stores):
  transport_co2_emissions = random.randint(low = 50000, high = 100000, size=(number_of_facilities))
  total_transport_co2_emissions.append(transport_co2_emissions.tolist())

algorithms = Algorithms()

num_iterations, number_of_generations, population_size, crossover_rate, show_plot = 100, 100, 100, 0.7, False

result_3_100_100_100_07 = algorithms.experiment(number_of_facilities, number_of_stores, fixed_costs_for_warehouses, 
                  number_of_cases_capacity, number_of_stores_capacity, demand_for_stores, 
                  transportation_cost, total_transport_co2_emissions, num_iterations,
                  number_of_generations, population_size, crossover_rate, show_plot)

Iteration :  0
Iteration :  1
Iteration :  2
Iteration :  3
Iteration :  4
Iteration :  5
Iteration :  6
Iteration :  7
Iteration :  8
Iteration :  9
Iteration :  10
Iteration :  11
Iteration :  12
Iteration :  13
Iteration :  14
Iteration :  15
Iteration :  16
Iteration :  17
Iteration :  18
Iteration :  19
Iteration :  20
Iteration :  21
Iteration :  22
Iteration :  23
Iteration :  24
Iteration :  25
Iteration :  26
Iteration :  27
Iteration :  28
Iteration :  29
Iteration :  30
Iteration :  31
Iteration :  32
Iteration :  33
Iteration :  34
Iteration :  35
Iteration :  36
Iteration :  37
Iteration :  38
Iteration :  39
Iteration :  40
Iteration :  41
Iteration :  42
Iteration :  43
Iteration :  44
Iteration :  45
Iteration :  46
Iteration :  47
Iteration :  48
Iteration :  49
Iteration :  50
Iteration :  51
Iteration :  52
Iteration :  53
Iteration :  54
Iteration :  55
Iteration :  56
Iteration :  57
Iteration :  58
Iteration :  59
Iteration :  60
Iteration :  61
Iteration :  62
It